# تطبيق توافق

- **Aim**: It is an verification system that ensures each ticket is used only by its rightful owner.

- **How it works**: The user logs in and registers their facial biometric data. When a ticket is purchased, it is linked to the facial data and a unique QR code is generated. At the event entrance, the QR code is scanned and facial matching is performed to allow access.
The system also allows ticket cancellation.




- **Therefore**: We wanna train a model that is able to read facial features and apply 1:1 varification between the user's image(s) on registration and the scanned image on event day that matches their ticket number.

# Methodology:


### 1. Insightface:
Is an open-source deep learning framework used in this project for face verification. It relies on ArcFace, a pre-trained model trained on large-scale facial datasets, to extract discriminative facial embeddings that uniquely represent each user. These embeddings are linked to ticket IDs and compared during verification to confirm whether the presented face matches the registered ticket owner, enabling accurate and reliable facial authentication under real-world conditions.

### 2. (1:1)  vs. (1:N) Verification:
In 1:1 verification, the face is matched only against the template linked to a specific ticket ID, while 1:N identification searches across all identities. We use 1:1 verification because the identity is already claimed by the ticket, making the process faster, more accurate, and more secure.


Install insight face,  onnxruntime and opencv

In [1]:
!pip -q install insightface onnxruntime opencv-python matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 140.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00


# Setting InsightFace - Face Analysis

Here, the InsightFace framework is initialized using a pre-trained ArcFace-based model. This component is responsible for detecting faces and extracting facial embeddings used for identity representation.

In [2]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis

app = FaceAnalysis(name="buffalo_l") #L
app.prepare(ctx_id=0, det_size=(640, 640)) # on GPU


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 92713.24KB/s] 
/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### Helper functions:

These helper functions handle basic image loading and preprocessing tasks, ensuring images are correctly read and formatted before being passed to the face analysis pipeline.

In [36]:
#1:
def read_image(path):
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    return img

#2: Cosine similarity is used to compare facial embeddings by measuring the angle between them.
#Higher similarity scores indicate a higher likelihood that both embeddings belong to the same individual.
def cosine_similarity(a, b):
    a = a / (np.linalg.norm(a) + 1e-12)
    b = b / (np.linalg.norm(b) + 1e-12)
    return float(np.dot(a, b))


In [37]:
# 3: How blurry is the image?
def laplacian_blur_score(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    return float(cv2.Laplacian(gray, cv2.CV_64F).var())

# 4: how bright is the image?
def brightness_score(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    return float(gray.mean())

#5: is the face near enough? by comparing size of bounding-box with the size of image.
def face_box_area_ratio(face, img_bgr):
    h, w = img_bgr.shape[:2]
    x1, y1, x2, y2 = face.bbox.astype(int)
    box_area = max(0, x2-x1) * max(0, y2-y1)
    return float(box_area) / float(h*w + 1e-12)

# 6: is the face frontal and reliable?
    """
    Rough yaw estimate from 5-point landmarks:
    If landmark order is [left_eye, right_eye, nose, left_mouth, right_mouth]
    yaw proxy = nose closer to left vs right eye distance.
    This is not perfect but works as a simple gate.
    """
def estimate_yaw_from_landmarks(face):

    if face.kps is None:
        return None
    kps = face.kps.astype(np.float32)
    left_eye, right_eye, nose = kps[0], kps[1], kps[2]
    dl = np.linalg.norm(nose - left_eye)
    dr = np.linalg.norm(nose - right_eye)
    if dl + dr == 0:
        return 0.0

    return float((dr - dl) / (dr + dl + 1e-12))


In [38]:
# The threshold of acceptance (acceptance critiria)
QUALITY_CFG = {
    "min_blur": 40.0,
    "min_brightness": 40.0,
    "max_brightness": 220.0,
    "min_face_area": 0.03,
    "max_abs_yaw": 0.35
}

#7: detect one face (here there is critira to accept and the user must provide clear picture of only thier face)
def detect_one_face(img_bgr):
    faces = app.get(img_bgr)
    if len(faces) == 0:
        return None, {"ok": False, "error": "No face detected"}
    if len(faces) > 1:
        return None, {"ok": False, "error": f"Multiple faces detected ({len(faces)}). Use one face only."}
    return faces[0], {"ok": True}


# new
# 8: detect any face then locate the largest and the nearest to the center being the face to be taken to compare and verify on (during event scan)
# in case there were multible faces showing
def detect_primary_face_for_verify(img_bgr, alpha=0.7, beta=0.3):

    faces = app.get(img_bgr)
    if len(faces) == 0:
        return None, {"ok": False, "error": "No face detected"}

    if len(faces) == 1:
        return faces[0], {"ok": True, "num_faces": 1, "picked": "only"}

    h, w = img_bgr.shape[:2]
    cx, cy = w / 2.0, h / 2.0

    #  area ratio of each face
    areas = np.array([face_box_area_ratio(f, img_bgr) for f in faces], dtype=np.float32)

    # center distance - near is better
    dists = []
    for f in faces:
        x1, y1, x2, y2 = f.bbox.astype(float)
        fx, fy = (x1 + x2) / 2.0, (y1 + y2) / 2.0
        d = (fx - cx) ** 2 + (fy - cy) ** 2
        dists.append(d)
    dists = np.array(dists, dtype=np.float32)

    # Normalize distances (bigger is better) --> turn into center score
    dist_norm = (dists - dists.min()) / (dists.max() - dists.min() + 1e-8)
    center_score = 1.0 - dist_norm

    # Final score
    score = alpha * areas + beta * center_score

    best_idx = int(np.argmax(score))
    best_face = faces[best_idx]

    return best_face, {
        "ok": True,
        "num_faces": len(faces),
        "picked": "hybrid_area_ratio+center",
        "best_idx": best_idx,
        "scores": score.tolist(),
        "areas": areas.tolist(),
        "center_score": center_score.tolist(),
    }


# new - limit bightness and blure on face area not on entire image (depending on bbox)
'''
def crop_face(img_bgr, face, pad=0.15):
    h, w = img_bgr.shape[:2]
    x1, y1, x2, y2 = face.bbox.astype(int)

    bw, bh = x2 - x1, y2 - y1
    px, py = int(bw * pad), int(bh * pad)

    x1 = max(0, x1 - px)
    y1 = max(0, y1 - py)
    x2 = min(w, x2 + px)
    y2 = min(h, y2 + py)

    return img_bgr[y1:y2, x1:x2]
'''
# worse performance with this function

# 9:
'''
 The quality_check function evaluates whether a detected face image is suitable
 for verification by assessing key quality factors such as image sharpness, brightness, face size, and pose.
 It ensures that only clear, well-aligned facial images are processed further,
 improving the reliability and accuracy of the face verification system.
'''
def quality_check(img_bgr, face, cfg=QUALITY_CFG):

    blur = laplacian_blur_score(img_bgr)
    bright = brightness_score(img_bgr)

    area = face_box_area_ratio(face, img_bgr)
    yaw = estimate_yaw_from_landmarks(face)


    reasons = []
    if blur < cfg["min_blur"]:
        reasons.append(f"Too blurry (blur={blur:.1f} < {cfg['min_blur']})")
    if bright < cfg["min_brightness"]:
        reasons.append(f"Too dark (brightness={bright:.1f} < {cfg['min_brightness']})")
    if bright > cfg["max_brightness"]:
        reasons.append(f"Too bright (brightness={bright:.1f} > {cfg['max_brightness']})")
    if area < cfg["min_face_area"]:
        reasons.append(f"Face too small (area={area:.3f} < {cfg['min_face_area']})")
    if yaw is not None and abs(yaw) > cfg["max_abs_yaw"]:
        reasons.append(f"Face too turned (yaw≈{yaw:.2f}, limit {cfg['max_abs_yaw']})")

    return {
        "ok": len(reasons) == 0,
        "blur": blur,
        "brightness": bright,
        "face_area_ratio": area,
        "yaw_proxy": yaw,
        "reasons": reasons
    }


### Single Face Detection - Enrollment
This function enforces strict face detection during enrollment, allowing only one visible face. This prevents incorrect ticket-to-face associations and ensures reliable identity registration.

In [39]:
ticket_db = {}
ticket_log = []

def enroll_ticket_v2(ticket_id, img_bgr, max_templates=3):

  # enroll so take one face

    # if not accepted from detect one face dor one of 2 reasons:
    face, det = detect_one_face(img_bgr)
    if not det["ok"]:
        ticket_log.append({"action":"enroll", "ticket_id":ticket_id, **det})
        return det

    # check quality and if not within critira
    q = quality_check(img_bgr, face)
    if not q["ok"]:
        out = {"ok": False, "error": "Quality check failed", **q}
        ticket_log.append({"action":"enroll", "ticket_id":ticket_id, **out})
        return out

   # take embeddings
    emb = face.embedding

    ticket_db.setdefault(ticket_id, [])
    if len(ticket_db[ticket_id]) < max_templates:
        ticket_db[ticket_id].append(emb)

    out = {"ok": True, "ticket_id": ticket_id, "templates": len(ticket_db[ticket_id]), **q}
    ticket_log.append({"action":"enroll", "ticket_id":ticket_id, **out})
    return out


### Verify 1:1 and calculate similarity between faces
During verification, this function selects the most relevant face when multiple faces are detected. It prioritizes the largest and most centrally located face to simulate real-world entry scenarios where bystanders may appear in the background.

In [40]:
# acceptance and review thresholds
DECISION_CFG = {
    "accept": 0.55,
    "review": 0.45
}

# Edited:
def verify_ticket_v2(ticket_id, img_bgr, decision_cfg=DECISION_CFG):

    if ticket_id not in ticket_db or len(ticket_db[ticket_id]) == 0:
        out = {"ok": False, "error": "Ticket not enrolled"}
        ticket_log.append({"action":"verify", "ticket_id":ticket_id, **out})
        return out

    # face, det = detect_one_face(img_bgr)
    face, det = detect_primary_face_for_verify(img_bgr, alpha=0.7, beta=0.3) # giving the alpha 0.7 for the largest face not the most centred face
    if not det["ok"]:
        ticket_log.append({"action":"verify", "ticket_id":ticket_id, **det})
        return det

    q = quality_check(img_bgr, face)
    if not q["ok"]:
        out = {"ok": False, "error": "Quality check failed", **q}
        ticket_log.append({"action":"verify", "ticket_id":ticket_id, **out})
        return out

    # In this step, the detected face is converted into a normalized facial embedding,
    # which serves as a compact numerical representation of the user’s identity.
    emb = face.embedding
    scores = [cosine_similarity(emb, e) for e in ticket_db[ticket_id]] # clcualte similarity distance
    best = max(scores)

    if best >= decision_cfg["accept"]:
        decision = "ACCEPT"
    elif best >= decision_cfg["review"]:
        decision = "REVIEW"
    else:
        decision = "REJECT"

    out = {
        "ok": True,
        "ticket_id": ticket_id,
        "best_similarity": float(best),
        "all_scores": [float(s) for s in scores],
        "decision": decision,
        **q
    }
    ticket_log.append({"action":"verify", "ticket_id":ticket_id, **out})
    return out


# Check the model

### 1. Upload image and enroll ticket
During enrollment, the facial embedding is securely linked to a unique ticket ID. This establishes a one-to-one relationship between the ticket and the ticket owner’s facial identity.

In [41]:
from google.colab import files
uploaded = files.upload()
paths = list(uploaded.keys())

ticket_id = "EVENT_TICKET_001"
for p in paths:
    img = read_image(p)
    print(p, "=>", enroll_ticket_v2(ticket_id, img, max_templates=3))


Saving Amani save.png to Amani save (2).png
Amani save (2).png => {'ok': True, 'ticket_id': 'EVENT_TICKET_001', 'templates': 1, 'blur': 792.7470571003869, 'brightness': 61.376929131769096, 'face_area_ratio': 0.12330966047280864, 'yaw_proxy': -0.0944068506360054, 'reasons': []}


### 2. Upload other image to compare and verify
uploaded same person with diff picture

In [42]:
uploaded = files.upload()
verify_path = list(uploaded.keys())[0]

imgv = read_image(verify_path)
verify_ticket_v2(ticket_id, imgv)


Saving Amani verify.jpg to Amani verify (2).jpg


{'ok': True,
 'ticket_id': 'EVENT_TICKET_001',
 'best_similarity': 0.6826301217079163,
 'all_scores': [0.6826301217079163],
 'decision': 'ACCEPT',
 'blur': 1251.6997583896289,
 'brightness': 108.66630952380953,
 'face_area_ratio': 0.12047619047619047,
 'yaw_proxy': 0.03021254390478134,
 'reasons': []}

best_similarity': 0.6826301217079163
The result shows a good similarity above threshold which is 55

In [43]:
ticket_log[-10:]

[{'action': 'enroll',
  'ticket_id': 'EVENT_TICKET_001',
  'ok': True,
  'templates': 1,
  'blur': 792.7470571003869,
  'brightness': 61.376929131769096,
  'face_area_ratio': 0.12330966047280864,
  'yaw_proxy': -0.0944068506360054,
  'reasons': []},
 {'action': 'verify',
  'ticket_id': 'EVENT_TICKET_001',
  'ok': True,
  'best_similarity': 0.6826301217079163,
  'all_scores': [0.6826301217079163],
  'decision': 'ACCEPT',
  'blur': 1251.6997583896289,
  'brightness': 108.66630952380953,
  'face_area_ratio': 0.12047619047619047,
  'yaw_proxy': 0.03021254390478134,
  'reasons': []}]

### 3. Test on custom built dataset of arabian features
The dtataset is collected to have 23 person registred and 23 other pics of them for verification


In [44]:
import os, zipfile, glob
from pathlib import Path

#
TRAIN_ZIP = "/content/Train-20251227T094751Z-1-001.zip"
TEST_ZIP  = "/content/verify-20251227T094749Z-1-001.zip"

DATA_DIR  = "/content/faces_dataset"
TRAIN_DIR = f"{DATA_DIR}/Train"
TEST_DIR  = f"{DATA_DIR}/verify"

train_paths = sorted(glob.glob(str(Path(TRAIN_DIR) / "*")))
test_paths = sorted(glob.glob(str(Path(TEST_DIR) / "*")))

print("Number of test images:", len(test_paths))
print("Number of train images:", len(train_paths))



os.makedirs(DATA_DIR, exist_ok=True)

with zipfile.ZipFile(TRAIN_ZIP, "r") as z:
    z.extractall(DATA_DIR)

with zipfile.ZipFile(TEST_ZIP, "r") as z:
    z.extractall(DATA_DIR)

print("Train dir:", TRAIN_DIR, "count:", len(glob.glob(TRAIN_DIR+"/*")))
print("Verify  dir:", TEST_DIR,  "count:", len(glob.glob(TEST_DIR+"/*")))
print("Sample train:", [Path(p).name for p in glob.glob(TRAIN_DIR+"/*")[:5]])
print("Sample test :", [Path(p).name for p in glob.glob(TEST_DIR+"/*")[:5]])


Number of test images: 23
Number of train images: 23
Train dir: /content/faces_dataset/Train count: 23
Verify  dir: /content/faces_dataset/verify count: 23
Sample train: ['Asala_Nasri.png', 'rayan_alahmari.png', 'Omar_Abdulrahman.png', 'Thunayyan_Khalid.png', 'Nasser_Alqasabi.png']
Sample test : ['Asala_Nasri.png', 'rayan_alahmari.png', 'Omar_Abdulrahman.png', 'Thunayyan_Khalid.png', 'Nasser_Alqasabi.png']


In [45]:
# Mapping names to ad ticket_Id:
# Removing spaces and underscores
import re
from pathlib import Path

def person_id_from_filename(fname: str) -> str:
    stem = Path(fname).stem
    stem = stem.strip()
    stem = re.sub(r"\s+", " ", stem)         # spaces
    stem = stem.replace(" _", " ").replace("_ ", " ")
    stem = stem.replace(" ", " ")            # unify to underscores to spaces
    stem = re.sub(r"_+", " ", stem)          # remove multiple underscores
    return stem.lower()


### Enrolling all faces to test, and viewing logs to see if ther picture matches acceptance critiria
During enrollment, the facial embedding is securely linked to a unique ticket ID. This establishes a one-to-one relationship between the ticket and the ticket owner’s facial identity.

In [46]:
import glob
import pandas as pd

#enrolling and acceptance critiria

# read_image, enroll_ticket_v2
# ticket_db = {} we already have it

#ُEnrolling them (test)

enroll_rows = []
train_paths = sorted(glob.glob(TRAIN_DIR + "/*"))

for p in train_paths:
    fname = Path(p).name
    pid = person_id_from_filename(fname)  # ticket_id = person name, to be changed
    img = read_image(p)
    out = enroll_ticket_v2(pid, img, max_templates=1)  # 1 template - cause we only have one pic
    enroll_rows.append({"file": fname, "person_id": pid, **out})

enroll_df = pd.DataFrame(enroll_rows)
enroll_df


,file,person_id,ok,ticket_id,templates,blur,brightness,face_area_ratio,yaw_proxy,reasons
0,Abdulmajeed_Abdullah.png,abdulmajeed abdullah,True,abdulmajeed abdullah,1,843.230335,90.010300,0.204611,0.000078,[]
1,Abdulrahman _Abumalih.png,abdulrahman abumalih,True,abdulrahman abumalih,1,208.914421,153.126441,0.099178,0.018844,[]
2,Adel_Emam.png,adel emam,True,adel emam,1,147.961572,89.714556,0.302678,-0.124804,[]
3,Ahmed_Alshugairi.png,ahmed alshugairi,True,ahmed alshugairi,1,172.666544,152.889529,0.175860,-0.154860,[]
4,Ahmed_Helmy.png,ahmed helmy,True,ahmed helmy,1,54.328469,71.980772,0.262012,0.155268,[]
5,Anas_Bukhash.png,anas bukhash,True,anas bukhash,1,41.667986,101.126764,0.271553,0.056091,[]
6,Asala_Nasri.png,asala nasri,True,asala nasri,1,126.770207,133.451906,0.186825,0.095732,[]
7,Elham_Ali.png,elham ali,True,elham ali,1,232.661979,107.854259,0.160201,0.013877,[]
8,Nasser_Alqasabi.png,nasser alqasabi,True,nasser alqasabi,1,164.713601,74.097195,0.186937,-0.066057,[]
9,Omar_Abdulrahman.png,omar abdulrahman,True,omar abdulrahman,1,222.663289,188.440866,0.355695,0.056670,[]


### False Acceptance Rate (FAR)
- is A person accepts B face?
-  is the percentage of impostor attempts that are incorrectly accepted by the system.

In [48]:
import random
import numpy as np

#get ppl in ticket_db
people = list(ticket_db.keys())

impostor_trials = []
random.seed(42)

for p in test_paths:
    fname = Path(p).name
    true_pid = person_id_from_filename(fname)
    img = read_image(p)

    # choose random ticket
    wrong_pid = random.choice([x for x in people if x != true_pid])

    out = verify_ticket_v2(wrong_pid, img)
    decision = out.get("decision", None)
    ok = out.get("ok", False)

    impostor_trials.append({
        "test_file": fname,
        "true_person": true_pid,
        "wrong_ticket": wrong_pid,
        "ok": ok,
        "decision": decision,
        "best_similarity": out.get("best_similarity", None)
    })

imp_df = pd.DataFrame(impostor_trials)
imp_valid = imp_df[imp_df["ok"] == True]

FAR = (imp_valid["decision"] == "ACCEPT").mean() * 100 if len(imp_valid) else 0
print(f"Impostor FAR (ACCEPT on wrong ticket): {FAR:.2f}%")

imp_df


Impostor FAR (ACCEPT on wrong ticket): 0.00%


,test_file,true_person,wrong_ticket,ok,decision,best_similarity
0,Abdulmajeed_Abdullah.png,abdulmajeed abdullah,rayan alahmari,True,REJECT,-0.021189
1,Abdulrahman _Abumalih.png,abdulrahman abumalih,ahmed alshugairi,True,REJECT,0.030317
2,Adel_Emam.png,adel emam,EVENT_TICKET_001,True,REJECT,0.016779
3,Ahmed_Alshugairi.png,ahmed alshugairi,nasser alqasabi,True,REJECT,-0.059986
4,Ahmed_Helmy.png,ahmed helmy,elham ali,True,REJECT,0.050535
5,Anas_Bukhash.png,anas bukhash,elham ali,True,REJECT,0.032034
6,Asala_Nasri.png,asala nasri,ahmed alshugairi,False,None,NaN
7,Elham_Ali.png,elham ali,adel emam,True,REJECT,-0.046365
8,Nasser_Alqasabi.png,nasser alqasabi,sara bugnah,True,REJECT,-0.028064
9,Omar_Abdulrahman.png,omar abdulrahman,bandar madkhali,True,REJECT,0.092343


### Verify:
This step compares the live facial embedding against the stored embedding associated with the ticket ID. A similarity score is computed to determine whether the presented face matches the registered ticket owner.

In [50]:
# check the result of 1:1 verfucation if same person (similarity) betwen enrolled(train data) and now (virify ticket: test data)


import pandas as pd
import glob
from pathlib import Path

verify_rows = []
test_paths = sorted(glob.glob(TEST_DIR + "/*"))

for p in test_paths:
    fname = Path(p).name
    true_pid = person_id_from_filename(fname)
    img = read_image(p)

    out = verify_ticket_v2(true_pid, img)  # same ticket

    verify_rows.append({
        "test_file": fname,
        "person_id": true_pid,
        "ok": out.get("ok", False),
        "decision": out.get("decision", None),
        "best_similarity": out.get("best_similarity", None),
        "error": out.get("error", None),
        "reasons": out.get("reasons", None),
    })

verify_df = pd.DataFrame(verify_rows)
verify_df


,test_file,person_id,ok,decision,best_similarity,error,reasons
0,Abdulmajeed_Abdullah.png,abdulmajeed abdullah,True,ACCEPT,0.717378,None,[]
1,Abdulrahman _Abumalih.png,abdulrahman abumalih,True,ACCEPT,0.673308,None,[]
2,Adel_Emam.png,adel emam,True,ACCEPT,0.602255,None,[]
3,Ahmed_Alshugairi.png,ahmed alshugairi,True,ACCEPT,0.574826,None,[]
4,Ahmed_Helmy.png,ahmed helmy,True,ACCEPT,0.675079,None,[]
5,Anas_Bukhash.png,anas bukhash,True,ACCEPT,0.770767,None,[]
6,Asala_Nasri.png,asala nasri,False,None,NaN,Quality check failed,[Too blurry (blur=29.8 < 40.0)]
7,Elham_Ali.png,elham ali,True,REVIEW,0.456171,None,[]
8,Nasser_Alqasabi.png,nasser alqasabi,True,ACCEPT,0.744772,None,[]
9,Omar_Abdulrahman.png,omar abdulrahman,True,ACCEPT,0.589255,None,[]


### GAR
This section evaluates the system’s performance on genuine verification attempts. Only test images that successfully passed the detection and quality checks (ok=True) are considered valid. The Genuine Accept Rate (GAR) is then calculated as the percentage of valid test samples that were correctly accepted (decision = ACCEPT) when verified against their corresponding ticke

In [51]:
valid = verify_df[verify_df["ok"] == True]
GAR = (valid["decision"] == "ACCEPT").mean() * 100 if len(valid) else 0

print(f"Usable test images (ok=True): {len(valid)}/{len(verify_df)}")
print(f"Genuine Accept Rate (GAR): {GAR:.2f}%")

Usable test images (ok=True): 19/23
Genuine Accept Rate (GAR): 94.74%


checks. The system achieved a Genuine Accept Rate (GAR) of 94.74%, indicating that the majority of valid test samples were correctly verified against their associated ticket IDs.
This result demonstrates strong reliability of the face verification pipeline under realistic conditions, with most failures attributed to image quality or detection constraints rather than incorrect identity matching.

# Other